In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time
from urllib.request import urlretrieve
from tqdm import tqdm
import os,csv

In [2]:
def driver_start():
    chrome_options = webdriver.ChromeOptions() 
    chrome_options.add_argument('--headless')#창없이 실행
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    #print(0)

    path='./chromedriver'
    driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
    driver.set_window_size(1024,1000)

    url = "https://www.yogiyo.co.kr/mobile/#/"
    driver.get(url)
    time.sleep(1)
    #print(1)
    driver.find_element_by_xpath('//*[@id="search"]/div/form/input').clear()
    driver.find_element_by_xpath('//*[@id="search"]/div/form/input').send_keys('서울특별시 동대문구 전농동 694 서울시립대학교')
    driver.find_element_by_xpath('//*[@id="button_search_address"]/button[2]').click()
    time.sleep(1)
    #print(2)

    driver.find_element_by_xpath('//*[@id="category"]/ul/li[5]/span').click() #치킨,피자등 카테고리 클릭
    time.sleep(1)
    #print(2)
    
    return driver

In [3]:
#있는 곳까지 스크롤
def scroll_element(shop,i):
    print(f"{i}th scroll")
    while True:
        try:
            some_tag = driver.find_element_by_xpath(f'//*[@title="{shop[i]}"]')
            if some_tag:
                break
        except Exception:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)

In [4]:
def stretch_review_page(review_cnt):
    click_count = int((review_cnt/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in range(click_count):
        try:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            driver.find_element_by_class_name('btn-more').click()
            time.sleep(2)
        except Exception as e:
            pass
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print('모든 리뷰 불러오기 완료!')

In [5]:
def craw(i,shop_in,driver,col_list):
    try:
        driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]').click()
    except Exception:
        driver.quit()
        driver=driver_start()
        time.sleep(3)
        driver.find_element_by_xpath(f'//*[@title="{shop_in[i]}"]').click()
        time.sleep(3)
        driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]').click()
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    a=soup.find_all("strong",{"class":"ng-binding"})
    star_tot=a[0].text.strip()
    review_cnt=int(a[1].text.strip())
    review_own_cnt=a[2].text
    shop=soup.find('span',{'ng-bind':'restaurant.name'}).text
    print(shop, review_cnt)
    
    if review_cnt==0: #리뷰없으면 종료
        print(0)
        return
    
    stretch_review_page(review_cnt)
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    b=soup.find_all('li',{'class':'list-group-item star-point ng-scope'})
    
    name_list,date_list,star_flavor_list,star_volume_list,star_deliver_list=[],[],[],[],[]
    menu_list,review_customer_list,review_owner_list,img_name_list=[],[],[],[]
    for i in tqdm(range(len(b)),desc='save..',mininterval=3):
        now=b[i]
        name=now.find('span',{'ng-show':"review.phone"}).text
        name_list.append(name)
        date=now.find('span',{'ng-bind':"review.time|since"}).text
        date_list.append(date)
        
        try:
            star=now.find_all('span',{'class':"points ng-binding"})
        except IndexError:
            print(f'{i}th review, none star')
            continue
            
        try:
            star_flavor=star[0].text
        except IndexError:
            star_flavor=None
        star_flavor_list.append(star_flavor)
        
        try:
            star_volume=star[1].text
        except IndexError:
            star_volume=None
        star_volume_list.append(star_volume)

        try:
            star_deliver=star[2].text
        except IndexError:
            star_deliver=None
            pass
        star_deliver_list.append(star_deliver)
            
        menu=now.find("div",{"class":"order-items default ng-binding"}).text.strip()
        menu_list.append(menu)
        review_customer=now.find("p",{"class":"ng-binding"}).text
        review_customer_list.append(review_customer)

    
        try:
            review_owner=now.find("p",{"ng-bind-html":"review.owner_reply.comment|strip_html"}).text
        except AttributeError:
            review_owner=None
        review_owner_list.append(review_owner)
        try:
            img_url=now.find('img')['src']
        except TypeError:
            img_name=None
        else:
            img_name=shop+'_'+str(i)
            path=f'./data/chicken/{shop}'
            if not os.path.isdir(path):
                os.mkdir(path)
            urlretrieve(img_url,path+'/'+img_name+'.jpg')
        img_name_list.append(img_name)
                
    #new=pd.DataFrame([[shop,name,date,star_flavor,star_volume,star_deliver,menu,review_customer,review_owner,img_name]],columns=col_list)
    data={'shop':shop,'name':name_list,'date':date_list,'star_flavor':star_flavor_list,'star_volume':star_volume_list,
         'star_deliver':star_deliver_list,'menu':menu_list,'review_customer':review_customer_list,
          'review_owner':review_owner_list,'img_name':img_name_list}
    new=pd.DataFrame(data)

    return new

In [6]:
fin_list=[1,1,1,1,1,1,1,1]
with open('./data/final_shop.csv', 'r', encoding='cp949') as f:
    rdr = csv.reader(f)
    for i,line in enumerate(rdr):
        fin_list[i] = line

In [10]:
shop=fin_list[0]
col_list=['shop','name','date','star_flavor','star_volume','star_deliver','menu','review_customer','review_owner','img_name']
#df=pd.DataFrame(columns=col_list)
start=172
#end=90
end=len(shop)

#driver.quit()
driver=driver_start()
for i in tqdm(range(start,end),desc='craw'):
    scroll_element(shop,i)
    
    time.sleep(3)
    try:
        driver.find_element_by_xpath(f'//*[@title="{shop[i]}"]').click()
    except Exception:
        driver.quit()
        driver=driver_start()
        time.sleep(2)
        scroll_element(shop,i)
        time.sleep(3)
        driver.find_element_by_xpath(f'//*[@title="{shop[i]}"]').click()
    time.sleep(3)
    

    new=craw(i,shop,driver,col_list)
    df=pd.concat([new,df])
    print(df.shape)
    driver.back()
    time.sleep(2)

print(f'tot_shop:{i},crawling sucess')

df.to_csv('./data/chicken5.csv')

<ipython-input-2-996b97e0e9c3>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
craw:   0%|          | 0/35 [00:00<?, ?it/s]

172th scroll
바사칸인생치킨 400
모든 리뷰 불러오기 시작...
모든 리뷰 불러오기 완료!



save..: 100%|██████████| 400/400 [00:20<00:00, 19.56it/s]


(1954, 10)


craw:   3%|▎         | 1/35 [01:54<1:04:59, 114.70s/it]

...

모든 리뷰 불러오기 완료!


save..: 100%|██████████| 17/17 [00:00<00:00, 31.14it/s]


(15589, 10)


craw: 100%|██████████| 35/35 [1:17:17<00:00, 132.50s/it]

tot_shop:206,crawling sucess


In [12]:
df.tail()

,shop,name,date,star_flavor,star_volume,star_deliver,menu,review_customer,review_owner,img_name
170,요녀석파스타,ky**님,2019년 6월 17일,5,5,3,"새우알리오 파스타/1,목살스테이크＋샐러드/1",제가 알리오 올리오를 엄청좋아해서\n항상해먹다가 언제부턴가 해먹기귀찮아서\n시켜먹기...,None,None
171,요녀석파스타,su**님,2019년 6월 13일,5,5,5,"커플세트（메인메뉴2＋에이드）/1(메뉴 선택(까르보나라 파스타),메뉴 선택2(차돌박이...",파스타 필라프 에이드 다 맛있어요! 배달도 빨리 옵니다!,None,None
172,요녀석파스타,ll**님,2019년 6월 8일,3,3,4,"커플세트（메인메뉴2＋에이드）/1(메뉴 선택(까르보나라 파스타),메뉴 선택2(까르보나...",로제 파스타가 로제 맛이아닌듯해요.... 배달이넘늦졌어요,None,None
173,요녀석파스타,wi**님,2019년 5월 21일,5,5,5,"도전세트（스테이크＋메인메뉴＋에이드）/1(메뉴 선택(목살필라프),에이드 메뉴 선택(딸...",고기가 맛있네요\n다음에는 파스타를 시킬 예정입니다,None,None
174,요녀석파스타,손님,2019년 5월 19일,5,5,5,"새우토마토 파스타/1,치킨텐더 샐러드/1",소스를 더 넣으주시면 좋겄어요～ 맛있게 만들어 주셨어 감사합니다～,None,요녀석파스타_174


In [ ]:
shop=fin_list[1]
col_list=['shop','name','date','star_flavor','star_volume','star_deliver','menu','review_customer','review_owner','img_name']
df=pd.DataFrame(columns=col_list)
start=100
#end=46
end=1en(shop)

driver=driver_start()
for i in tqdm(range(start,end),desc='craw'):
    scroll_element(shop,i)
    
    time.sleep(3)
    try:
        driver.find_element_by_xpath(f'//*[@title="{shop[i]}"]').click()
    except Exception:
        driver.quit()
        driver=driver_start()
        time.sleep(2)
        scroll_element(shop,i)
        time.sleep(3)
        driver.find_element_by_xpath(f'//*[@title="{shop[i]}"]').click()
    time.sleep(3)
    

    new=craw(i,shop,driver,col_list)
    df=pd.concat([new,df])
    print(df.shape)
    driver.back()
    time.sleep(2)

print(f'tot_shop:{i},crawling sucess')

df.to_csv('./data/pizza2.csv')

In [18]:
shop[40]

'라와마라탕-3호점'

In [9]:
df.to_csv('./data/pig2.csv')